In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
path='/content/gdrive/MyDrive/MachineLearning/SVM'
os.chdir(path)

Target based Encoding of the categorical attributes

In [ ]:
#to read the data
import pandas as pd
import numpy as np
#for data preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
# for modelling
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC,SVR
# Performance Metrics
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
#read the data
df=pd.read_csv('UnivBank.csv', na_values=['?','#'])

In [ ]:
df.head()
df.shape

(5000, 14)

In [ ]:
df.tail()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
4995,4996,29,3,40,92697,1,1.9,3,0.0,0,0.0,0.0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85.0,0,0.0,0.0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0.0,0,0.0,0.0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0.0,0,0.0,0.0,1,0
4999,5000,28,4,83,92612,3,0.8,1,0.0,0,0.0,0.0,1,1


In [ ]:
 df.describe()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,4998.000000,5000.000000,4998.000000,4999.000000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93152.503000,2.396400,1.937938,1.881000,56.521409,0.096000,0.104442,0.060412,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,2121.852197,1.147663,1.747659,0.839869,101.727873,0.294621,0.305863,0.238273,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,9307.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df.isnull().sum()

ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              2
Personal Loan         0
Securities Account    2
CD Account            1
Online                0
CreditCard            0
dtype: int64

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df.loc[:,df.columns!='Personal Loan'],df['Personal Loan'],test_size=0.30,random_state=100)

In [ ]:
X_train.shape
# X_test.shape

(3500, 13)

In [ ]:
y_train.shape

(3500,)

In [ ]:
y_test.shape

(1500,)

In [ ]:
y_train.value_counts(normalize=True)*100

0    90.8
1     9.2
Name: Personal Loan, dtype: float64

In [ ]:
X_train.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Securities Account', 'CD Account', 'Online',
       'CreditCard'],
      dtype='object')

In [ ]:
#converting numeric to categorical
dropcols=['ID','ZIP Code']
catcols=['Family','Education', 'Securities Account', 'CD Account', 'Online',
       'CreditCard']
X_train[catcols]=X_train[catcols].astype('category')
X_train.dtypes
y_train=y_train.astype('category')
y_test=y_test.astype('category')

In [ ]:
X_test[catcols]=X_test[catcols].astype('category')
X_test.dtypes

ID                       int64
Age                      int64
Experience               int64
Income                   int64
ZIP Code                 int64
Family                category
CCAvg                  float64
Education             category
Mortgage               float64
Securities Account    category
CD Account            category
Online                category
CreditCard            category
dtype: object

In [ ]:
X_train.isnull().sum()

ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              1
Securities Account    1
CD Account            1
Online                0
CreditCard            0
dtype: int64

In [ ]:
#dropping the unnecessary columns
X_train=X_train.drop(dropcols,axis=1)
X_test=X_test.drop(dropcols,axis=1)

In [ ]:
#imputing na values
imputeMortgage=SimpleImputer(strategy='mean')
imputeSec_CD=SimpleImputer(strategy='most_frequent')

In [ ]:
#separating numeric and categorical
X_train_num=X_train.drop(catcols,axis=1)
X_train_cat=X_train[catcols]

In [ ]:
X_test_num=X_test.drop(catcols,axis=1)
X_test_cat=X_test[catcols]

In [ ]:
#Imputing the train data
X_train_num=pd.DataFrame(imputeMortgage.fit_transform(X_train_num),columns=X_train_num.columns)
X_train_cat=pd.DataFrame(imputeSec_CD.fit_transform(X_train_cat),columns=X_train_cat.columns)

In [ ]:
X_train.isnull().sum()

Age                   0
Experience            0
Income                0
Family                0
CCAvg                 0
Education             0
Mortgage              1
Securities Account    1
CD Account            1
Online                0
CreditCard            0
dtype: int64

In [ ]:
X_train_num.isna().sum()

Age           0
Experience    0
Income        0
CCAvg         0
Mortgage      0
dtype: int64

In [ ]:
#imputing the test data
X_test_num=pd.DataFrame(imputeMortgage.transform(X_test_num),columns=X_test_num.columns)
X_test_cat=pd.DataFrame(imputeSec_CD.transform(X_test_cat),columns=X_test_cat.columns)

In [ ]:
X_train_num.isna().sum()

Age           0
Experience    0
Income        0
CCAvg         0
Mortgage      0
dtype: int64

In [ ]:
X_train_cat.isnull().sum()

Family                0
Education             0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

In [ ]:
X_test_num

,Age,Experience,Income,CCAvg,Mortgage
0,29.0,3.0,31.0,0.3,0.0
1,59.0,34.0,60.0,2.1,234.0
2,58.0,33.0,23.0,0.2,0.0
3,54.0,29.0,34.0,0.1,0.0
4,56.0,31.0,11.0,0.2,90.0
...,...,...,...,...,...
1495,50.0,24.0,155.0,7.3,0.0
1496,34.0,9.0,134.0,4.6,164.0
1497,52.0,26.0,158.0,3.7,251.0
1498,60.0,35.0,122.0,1.3,0.0


In [ ]:
#Standardizing the numeric data
sd=StandardScaler()
X_train_num=pd.DataFrame(sd.fit_transform(X_train_num),columns=X_train_num.columns)
X_test_num=pd.DataFrame(sd.transform(X_test_num),columns=X_test_num.columns)

In [ ]:
#Encoding the categorical attributes
ohe=OneHotEncoder(handle_unknown='ignore')
# column=ohe.get_feature_names(X_train_cat.columns
X_train_cat=pd.DataFrame(ohe.fit_transform(X_train_cat).todense(),columns=ohe.get_feature_names_out())



In [ ]:
X_train_cat.head()

,Family_1.0,Family_2.0,Family_3.0,Family_4.0,Education_1.0,Education_2.0,Education_3.0,Securities Account_0.0,Securities Account_1.0,CD Account_0.0,CD Account_1.0,Online_0.0,Online_1.0,CreditCard_0.0,CreditCard_1.0
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [ ]:
X_test_cat=pd.DataFrame(ohe.transform(X_test_cat).todense(),columns=ohe.get_feature_names_out())

In [ ]:
Xtrain=pd.concat([X_train_num,X_train_cat],axis=1)
print(Xtrain.shape)
# y_train.shape
Xtrain.dtypes
print(X_test.shape)

(3500, 20)
(1500, 11)


In [ ]:
Xtest=pd.concat([X_test_num,X_test_cat],axis=1)

In [ ]:
#Model building
model=SVC()
param_grid={'C':[1.0,1.5,2.0], 'kernel':['rbf','sigmoid','poly'], 'gamma': [1.0,0.1,0.01]}
search=GridSearchCV(model,param_grid,cv=5)
search.fit(Xtrain,y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1.0, 1.5, 2.0], 'gamma': [1.0, 0.1, 0.01],
                         'kernel': ['rbf', 'sigmoid', 'poly']})

In [ ]:
print(search.best_params_)

{'C': 2.0, 'gamma': 0.1, 'kernel': 'rbf'}


In [ ]:
model=SVC(C=2.0,kernel='rbf',gamma=0.1)
svc=model.fit(Xtrain,y_train)
svctest=model.predict(Xtest)
svctrain=model.predict(Xtrain)

In [ ]:
confusion_matrix(svctest,y_test)

array([[1340,   27],
       [   2,  131]])

In [ ]:


confusion_matrix(svctrain,y_train)

array([[3176,   40],
       [   2,  282]])

In [ ]:
print(classification_report(svctest,y_test))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1367
           1       0.83      0.98      0.90       133

    accuracy                           0.98      1500
   macro avg       0.91      0.98      0.94      1500
weighted avg       0.98      0.98      0.98      1500



In [ ]:
print(classification_report(svctrain,y_train))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3216
           1       0.88      0.99      0.93       284

    accuracy                           0.99      3500
   macro avg       0.94      0.99      0.96      3500
weighted avg       0.99      0.99      0.99      3500



In [ ]:
import pickle as pkl
pkl.dump(Xtrain, open('svcXtrain.pkl', 'wb'))
pkl.dump(Xtest,open('svcXtest.pkl','wb'))
pkl.dump(y_train,open('svcytrain.pkl','wb'))
pkl.dump(y_test,open('svcytest.pkl','wb'))
pkl.dump(svc,open('svc.pkl','wb'))